In [1]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

pd.set_option('max_colwidth', 200)
pd.options.display.max_rows = 200

from const import *
import utils, predict, train_model_2

/home/damonhao/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
def check_col(col_name, df_val, df_test):
    print 'val'
    print df_val[col_name].describe()
    print 'null pcet: ', df_val[col_name].isnull().sum() / float(df_val.shape[0])
    print 'test'
    print df_test[col_name].describe()
    print 'null pcet: ', df_test[col_name].isnull().sum() / float(df_test.shape[0])

In [3]:
lgbm = utils.load_model('Booster_bagging_fraction:0.85_bagging_freq:3_categorical_column:[0, 1, 2]_feature_fraction:0.7_learning_rate:0.04_max_bin:255_max_depth:8_num_leaves:200_08-13-12-16-48.pkl')

In [4]:
df_lgbm_f_gain = utils.get_lgbm_feature_score(lgbm.feature_name(), lgbm.feature_importance('gain'))
df_lgbm_f_gain

,features,scores
0,user_product_bought_times/user_order_count,3.569620e+06
1,order_number-user_product_order_number_max,2.489618e+06
2,order_streak,2.107934e+06
3,product_id,1.451767e+06
4,user_product_bought_times/user_product_order_number_max-user_product_order_number_min,9.254021e+05
5,user_product_order_number_max/user_order_number_max,6.543175e+05
6,aisle_id,5.863893e+05
7,last,5.522081e+05
8,prev1,3.876577e+05
9,product_reorder_ratio,3.610845e+05


In [5]:
df_test_core = pd.read_csv('../input/test_diff_order_streaks_fix_add_new_core.csv')

In [8]:
df_val_core = pd.read_csv('../input/validate_diff_order_streaks_fix_add_new_core.csv')

In [35]:
df_test_core.shape

(4833292, 143)

In [36]:
df_val_core.shape

(1693004, 146)

In [37]:
print df_val_core.columns[~df_val_core.columns.isin(df_test_core.columns)]

Index([u'label', u'department_bought_times',
       u'user_product/user_department_bought_times'],
      dtype='object')


In [34]:
check_col('user_department_reorder_ratio', df_val_core, df_test_core)

val
count    1.693004e+06
mean     4.362213e-01
std      2.658038e-01
min      0.000000e+00
25%      2.307692e-01
50%      4.666667e-01
75%      6.569343e-01
max      9.880952e-01
Name: user_department_reorder_ratio, dtype: float64
null pcet:  0.0
test
count    4.833292e+06
mean     4.377058e-01
std      2.653829e-01
min      0.000000e+00
25%      2.380952e-01
50%      4.666667e-01
75%      6.585366e-01
max      9.893617e-01
Name: user_department_reorder_ratio, dtype: float64
null pcet:  0.0


In [9]:
df_test_core['user_product_bought_times/user_order_count'].describe()

count    4.833292e+06
mean     1.545985e-01
std      1.648642e-01
min      1.010101e-02
25%      4.545455e-02
50%      9.803922e-02
75%      2.000000e-01
max      1.000000e+00
Name: user_product_bought_times/user_order_count, dtype: float64

In [10]:
df_val_core['user_product_bought_times/user_order_count'].describe()

count    1.693004e+06
mean     1.548367e-01
std      1.644308e-01
min      1.010101e-02
25%      4.545455e-02
50%      9.756098e-02
75%      2.000000e-01
max      1.000000e+00
Name: user_product_bought_times/user_order_count, dtype: float64

In [4]:
df_test_diff[df_test_diff['user_product_bought_times/user_product_order_number_max-user_product_order_number_min'].isin([np.inf, -np.inf])].shape

(2899174, 142)

In [ ]:
df_test_diff

In [5]:
df_val_diff = pd.read_csv('../input/validate_diff_order_streaks.csv')

In [6]:
a = df_val_diff[df_val_diff['user_product_bought_times/user_product_order_number_max-user_product_order_number_min'].isin([np.inf, -np.inf])]

In [7]:
a.shape

(1017667, 143)

In [12]:
df_test_diff.user_active_days.describe()

count    4.833292e+06
mean     2.797156e+03
std      2.116562e+03
min      0.000000e+00
25%      1.199000e+03
50%      2.316000e+03
75%      3.860000e+03
max      2.151400e+04
Name: user_active_days, dtype: float64

In [8]:
(df_val_diff['user_product_order_number_max'] == df_val_diff['user_product_order_number_min']).sum()

1017667

In [10]:
df_val_diff.order_streak.isnull().sum() / float(df_val_diff.shape[0])

0.49263675691256487

In [13]:
df_val_diff.user_active_days.describe()

count    1.693004e+06
mean     2.791833e+03
std      2.115767e+03
min      0.000000e+00
25%      1.194000e+03
50%      2.309000e+03
75%      3.886000e+03
max      1.994400e+04
Name: user_active_days, dtype: float64

In [20]:
df_1 = pd.read_csv('../output/submit_max_f1_08-13-17-14-43.csv')

In [21]:
df_1.head()

,user_id,order_id,products
0,165002,367533,13176 21137 39877 33135 20379 19057 8174 37317 21903 42244 47209 38995 48679 47626 26862 25879
1,165004,1732315,1463 13517
2,165005,2323336,13176 7252 20738 42282 47402 21288 26348 38928 12341 21903 3298 16953 907 24135
3,165007,293711,28987 1217 10839 44765 23288
4,165008,1499884,None 48733 12519


In [24]:
df_2 = predict.read_max_f1_analysis('../prediction/analysis_08-13-17-04-28_validate_core_lgbm_08-13-11-00-07.pkl')

In [26]:
df_2[df_2.user_id.isin(df_1.head()[UID])]

,user_id,true_products,products,scores
5469,165002,"[13176, 21137, 39877, 33135, 20379, 19057, 8174, 42244, 47209, 38995, 29675, 27966, 11068, 4562, 41319, 36650, 35887, 15683, 1935, 17794, 14633]","[13176, 21137, 39877, 33135, 20379, 19057, 8174, 37317, 21903, 42244, 47209, 38995, 48679, 47626, 26862, 25879]",0.540541
5708,165005,"[13176, 20738, 26348, 24135, 22507, 16797, 47498, 13032, 30588, 27796, 9434, 13424, 17313, 20955, 22178, 41580]","[13176, 7252, 20738, 42282, 47402, 21288, 26348, 38928, 12341, 21903, 3298, 16953, 907, 24135]",0.266667
13918,165007,"[28987, 20327]","[28987, 1217, 10839, 44765, 23288]",0.285714
14805,165004,"[1463, 13517]","[1463, 13517]",1.000000
25984,165008,"[2078, 27796, 33953]","[None, 48733, 12519]",0.000000


In [2]:
df_submit = pd.read_csv('../output/submit_max_f1_08-13-18-22-50.csv')

In [3]:
df_submit.shape

(75000, 2)